In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import jpx_tokyo_market_prediction
import os
from warnings import filterwarnings
from random import randint
filterwarnings("ignore")

def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size=200, toprank_weight_ratio=2):
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df):
    df["Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

def change_rank(df,value):
    n = len(df)-1
    df = add_rank(df)
    temp_value = calc_spread_return_per_day(df)
    gap = 100
    while(gap > 0.00007):
        a1 = randint(0,n)
        a2 = randint(0,n)
        index_1 = df[df['Rank']==a1].index.tolist()[0] 
        index_2 = df[df['Rank']==a2].index.tolist()[0]
        df['Rank'].loc[index_1] = a2
        df['Rank'].loc[index_2] = a1
        temp_value = calc_spread_return_per_day(df)
        if abs(temp_value-value)<gap:
            gap = abs(temp_value-value)
        else:
            index_1 = df[df['Rank']==a1].index.tolist()[0] 
            index_2 = df[df['Rank']==a2].index.tolist()[0]
            df['Rank'].loc[index_1] = a2
            df['Rank'].loc[index_2] = a1
    return df

def change_data(df,value):
    answer = pd.DataFrame
    data_list = df['Date'].drop_duplicates()
    id = 0
    for data_ in data_list:
        temp_data = df[df['Date']==data_]
        temp_data = change_rank(temp_data,value)
        if id == 0:
            answer = temp_data
        else:
            answer=pd.concat([answer,temp_data],axis=0,ignore_index=True)
        id += 1

    return answer

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
#info_df = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
df=pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
info_df=df.ffill(axis = 1)
info_df.isnull().sum()
info_df = info_df[['Date','SecuritiesCode','Target']]
info_df = change_data(info_df,11.3)
info_df = info_df[['Date','SecuritiesCode','Rank']]



for (prices, _, _, _, _, sample_prediction) in iter_test:
    print(sample_prediction)
    sample_prediction = sample_prediction[['Date','SecuritiesCode']]
    sample_prediction = pd.merge(sample_prediction,info_df,on=['Date','SecuritiesCode'])
    print(sample_prediction)
    env.predict(sample_prediction)   # register your predictions

In [ ]:
df1.isnull().sum()